In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm

### Read the data

In [ ]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

test_fn2 = 'day' + str(day) + 'test2.txt'

test_file_content2 = open(test_fn2).read().split('\n')

### Part 1

In [ ]:
class Machine:
    def __init__(self):
        self.modules = list()

    def press_button(self):
        any_moved_sand = False
        # pulses is a queue of pulses
        button = BroadcastModule('button')
        button.add_destination_module(self.modules[0])
        init_pulse = Pulse(button, self.modules[0], 'low')

        pulses_history = [init_pulse]

        first_module = self.get_module('broadcaster')
        pulses, moved_sand = first_module.process_pulse(init_pulse)
        if moved_sand:
            any_moved_sand = True

        while True:
            if len(pulses) == 0:
                break
            pulse = pulses.pop(0)
            pulses_history.append(pulse)
            new_pulses, moved_sand = pulse.receiver.process_pulse(pulse)
            if moved_sand:
                any_moved_sand = True
            if new_pulses and len(new_pulses) > 0:
                pulses.extend(new_pulses)


        n_high = sum([1 for pulse in pulses_history if pulse.get_level() == 'high'])
        n_low = sum([1 for pulse in pulses_history if pulse.get_level() == 'low'])

        return n_high, n_low, any_moved_sand


    def add_module(self, module):
        self.modules.append(module)

    def get_module(self, name):
        for module in self.modules:
            if module.name == name:
                return module
            
    def add_sender_receiver_relation(self, sender, receiver):
        # check if receiver is in the list of modules
        if receiver not in [module.name for module in self.modules]:
            receiver_module = BroadcastModule(receiver)
        else:
            receiver_module = self.get_module(receiver)
        sender_module = self.get_module(sender)
        
        sender_module.add_destination_module(receiver_module)
        receiver_module.add_origin_module(sender_module)

    def initialise_conjunctions(self):
        for module in self.modules:
            if module.get_type() == '&':
                module.initialise_pulse_levels()

    def __repr__(self):
        return_str = ''
        for module in self.modules:
            return_str += str(module) + ' -> '
            for dest_module in module.destination_modules:
                return_str += str(dest_module) + ', '
            return_str += '\n'

        return return_str

class Module:
    def __init__(self, name):
        self.name = name
        self.destination_modules = list()
        self.origin_modules = list()

    def add_destination_module(self, module):
        self.destination_modules.append(module)

    def add_origin_module(self, module):
        self.origin_modules.append(module)

    def process_pulse(self, pulse):
        receiver_name = pulse.receiver.get_name()
        pulse_level = pulse.get_level()
        if receiver_name == 'rx' and pulse_level == 'low':
            return self.process_pulse_module(pulse), True
        else:
            return self.process_pulse_module(pulse), False

    def get_name(self):
        return self.name

    def __repr__(self):
        return self.name


class BroadcastModule(Module):
    def __init__(self, name):
        super().__init__(name)

    def get_type(self):
        return 'bc'
    
    def process_pulse_module(self, pulse):
        return [Pulse(self, module, pulse.get_level()) for module in self.destination_modules]

    def __repr__(self):
        return self.name


class ConjunctionModule(Module):
    def __init__(self, name):
        super().__init__(name)

        self.most_recent_pulse_level = dict()

    def initialise_pulse_levels(self):
        for module in self.origin_modules:
            self.most_recent_pulse_level[module.name] = 'low'

    def get_type(self):
        return '&'

    def process_pulse_module(self, pulse):
        sender_name = pulse.get_sender_name()
        self.most_recent_pulse_level[sender_name] = pulse.get_level()

        if all(level == 'high' for level in self.most_recent_pulse_level.values()):
            return [Pulse(self, module, 'low') for module in self.destination_modules]
        else:
            return [Pulse(self, module, 'high') for module in self.destination_modules]

    def __repr__(self):
        return '&' + self.name


class FlipFlopModule(Module):
    def __init__(self, name):
        super().__init__(name)
        self.status = 'off'

    def get_type(self):
        return '%'

    def process_pulse_module(self, pulse):
        if pulse.get_level() == 'high':
            return
        if self.status == 'off':
            self.status = 'on'
            return_list = list()
            for module in self.destination_modules:
                return_list.append(Pulse(self, module, 'high'))
        else:
            self.status = 'off'
            return_list = list()
            for module in self.destination_modules:
                return_list.append(Pulse(self, module, 'low'))
        return return_list


    def __repr__(self):
        return '%' + self.name


class Pulse:
    def __init__(self, sender=None, receiver=None, level='low'):
        self.sender = sender
        self.level = level
        self.receiver = receiver

    def get_level(self):
        return self.level
    
    def set_level(self, level):
        self.level = level

    def get_sender_name(self):
        return self.sender.get_name()
    
    def __repr__(self):
        return self.sender.get_name() + ' -' + self.get_level() + '-> ' + self.receiver.get_name()


In [ ]:
working_file = file_content

machine = Machine()
for row in working_file:
    sender = row.split(' -> ')[0]
    if sender.startswith('&'):
        machine.add_module(ConjunctionModule(sender[1:]))
    elif sender.startswith('%'):
        machine.add_module(FlipFlopModule(sender[1:]))
    else:
        machine.add_module(BroadcastModule(sender))

for row in working_file:
    sender = row.split(' -> ')[0]
    sender = sender.replace('&', '').replace('%', '')
    receivers = row.split(' -> ')[1].split(', ')
    for receiver in receivers:
        machine.add_sender_receiver_relation(sender, receiver)


machine.initialise_conjunctions()

total_n_high = 0
total_n_low = 0

for button_press in tqdm(range(1000)):
    n_high, n_low, _ = machine.press_button()
    total_n_high += n_high
    total_n_low += n_low

print(total_n_high*total_n_low)




### Part 2

In [ ]:
working_file = file_content

machine = Machine()
for row in working_file:
    sender = row.split(' -> ')[0]
    if sender.startswith('&'):
        machine.add_module(ConjunctionModule(sender[1:]))
    elif sender.startswith('%'):
        machine.add_module(FlipFlopModule(sender[1:]))
    else:
        machine.add_module(BroadcastModule(sender))

for row in working_file:
    sender = row.split(' -> ')[0]
    sender = sender.replace('&', '').replace('%', '')
    receivers = row.split(' -> ')[1].split(', ')
    for receiver in receivers:
        machine.add_sender_receiver_relation(sender, receiver)


machine.initialise_conjunctions()

total_n_high = 0
total_n_low = 0

for button_press in tqdm(range(1, 212986464842911 + 1)):
    n_high, n_low, any_moved_sand = machine.press_button()
    total_n_high += n_high
    total_n_low += n_low

    if any_moved_sand:
        print(f'sand moved after {button_press} button presses')
        break

print(total_n_high*total_n_low)


